# Pakete prüfen und laden
for (pkg in c("tidygeocoder", "dplyr", "terra", "stringr", "curl")) {
  if (!requireNamespace(pkg, quietly = TRUE)) install.packages(pkg)
  library(pkg, character.only = TRUE)
}

# -----------------------------
# FUNKTION: meine_funktion()
# -----------------------------
meine_funktion <- function(p_adresse, 
                           p_save_path, 
                           p_buffer_1 = 200, 
                           p_buffer_2 = 50) {
  
  # Adresse → Koordinaten
  l_adresse <- p_adresse %>%
    tidygeocoder::geocode(address = adresse, method = "osm")
  
  v <- terra::vect(l_adresse, geom = c("long", "lat"), crs = "EPSG:4326")
  v_utm <- terra::project(v, "EPSG:25832")
  v_gk3 <- terra::project(v, "EPSG:31467")
  
  # Koordinaten vorbereiten
  coords <- terra::geom(v_utm)
  x_val <- as.numeric(substr(coords[1, "x"], 1, 3))
  y_val <- as.numeric(substr(coords[1, "y"], 1, 4))
  
  # DGM-Kachel-Link erzeugen
  l_url <- paste0(
    "https://opengeodata.lgl-bw.de/data/dgm/dgm1_32_",
    x_val, "_", y_val, "_2_bw.zip"
  )
  destfile <- file.path(p_save_path, paste0(x_val, "_", y_val, "_2_bw.zip"))
  
  # Existenz prüfen
  check <- tryCatch({
    curl::curl_fetch_memory(l_url)
    TRUE
  }, error = function(e) FALSE)
  if (!check) {
    message("❌ Kachel existiert nicht: ", l_url)
    return(NULL)
  }
  
  # Download
  download.file(l_url, destfile = destfile, mode = "wb")
  message("✅ Datei heruntergeladen: ", destfile)
  
  # Entpacken
  unzip_success <- tryCatch({
    unzip(destfile, exdir = p_save_path)
    TRUE
  }, error = function(e) {
    message("❌ Fehler beim Entpacken: ", destfile)
    FALSE
  })
  if (!unzip_success) return(NULL)
  message("📦 Datei entpackt")
  
  # Entpackte .xyz-Dateien suchen
  unzipped_folder <- list.dirs(p_save_path, full.names = TRUE, recursive = FALSE)
  unzipped_folder <- unzipped_folder[grep("dgm1_32_", unzipped_folder)]
  if (length(unzipped_folder) == 0) {
    message("⚠️ Keine entpackten DGM-Ordner gefunden.")
    return(NULL)
  }
  
  p_xyz_files <- list.files(unzipped_folder[1], pattern = "\\.xyz$", full.names = TRUE)
  if (length(p_xyz_files) == 0) {
    message("⚠️ Keine .xyz-Dateien gefunden.")
    return(NULL)
  }
  
  # XYZ-Dateien einlesen
  xyz_list <- lapply(p_xyz_files, function(f) {
    tryCatch({
      read.table(f, header = FALSE)
    }, error = function(e) NULL)
  })
  xyz_list <- xyz_list[!sapply(xyz_list, is.null)]
  
  dgm_df_all <- do.call(rbind, xyz_list)
  colnames(dgm_df_all) <- c("x", "y", "z")
  message("✅ XYZ-Dateien zusammengeführt.")
  
  # Raster erstellen und umprojizieren
  dgm <- terra::rast(dgm_df_all, type = "xyz", crs = "EPSG:25832")
  dgm <- terra::project(dgm, "EPSG:31467")
  
  # Hilfsfunktion: Zoom um Punkt
  zoom_to_point <- function(raster, point, buffer = 200, col_point = "red") {
    pt <- terra::geom(point)
    if (is.vector(pt)) {
      x <- pt[1]; y <- pt[2]
    } else {
      x <- pt[1, "x"]; y <- pt[1, "y"]
    }
    xlim <- c(x - buffer, x + buffer)
    ylim <- c(y - buffer, y + buffer)
    plot(raster, xlim = xlim, ylim = ylim)
    plot(point, add = TRUE, pch = 16, col = col_point, cex = 2)
  }
  
  # Plots
  par(mfrow = c(1, 3))
  plot(dgm, main = "Gesamtes DGM")
  plot(v_gk3, add = TRUE, pch = 16, col = "red")
  zoom_to_point(dgm, v_gk3, buffer = p_buffer_1)
  zoom_to_point(dgm, v_gk3, buffer = p_buffer_2)
  
  message("🎉 Fertig! Alles durchgelaufen.")
  return(list(raster = dgm, punkt = v_gk3))
}